In [1]:
pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer
import math 
import torch

device = torch.device("cpu")
if torch.cuda.is_available():
 device = torch.device("cuda")
device

c:\Users\Hallgrimur\anaconda3\lib\site-packages\torch\cuda\__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [2]:
#Load data
dataset = load_dataset("copenlu/answerable_tydiqa")
dataset

Using custom data configuration copenlu--nlp_course_tydiqa-cceecfb5416d988a
Found cached dataset parquet (C:/Users/Hallgrimur/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-cceecfb5416d988a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 116067
    })
    validation: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 13325
    })
})

In [3]:
#split data 
train_set = dataset["train"]
validation_set = dataset["validation"]

train_set = train_set.to_pandas()
validation_set = validation_set.to_pandas()

In [ ]:
#preprocess data

eng_train_set = train_set[train_set['language'] == 'english']
fin_train_set = train_set[train_set['language'] == 'finnish']
jap_train_set = train_set[train_set['language'] == 'japanese']

eng_validation_set = validation_set[validation_set['language'] == 'english']
fin_validation_set = validation_set[validation_set['language'] == 'finnish']
jap_validation_set = validation_set[validation_set['language'] == 'japanese']

fin_train_set

,question_text,document_title,language,annotations,document_plaintext,document_url
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort
10,Mitä on altruismi?,Altruismi,finnish,"{'answer_start': [44], 'answer_text': ['epäits...","\n\n\nAltruismi ([1], ”toinen”[2]) tarkoittaa...",https://fi.wikipedia.org/wiki/Altruismi
12,Mikä oli Wilhelm Wagner viimeinen sävellys?,Richard Wagner,finnish,"{'answer_start': [597], 'answer_text': ['Parsi...",Wagnerin mestariteoksia ovat hänen myöhäiskaud...,https://fi.wikipedia.org/wiki/Richard%20Wagner
13,Missä Harz sijaitsee?,Harz,finnish,"{'answer_start': [25], 'answer_text': ['Pohjoi...",\n\nHarz on horstivuoristo Pohjois-Saksassa[1]...,https://fi.wikipedia.org/wiki/Harz
15,Miten paljon täysikasvuinen ahma painaa?,Ahma,finnish,"{'answer_start': [173], 'answer_text': ['Naara...","Ahmaa tavataan Suomessa, Ruotsissa, Norjassa, ...",https://fi.wikipedia.org/wiki/Ahma
...,...,...,...,...,...,...
116005,Kuinka leveä Iso-Kukkanen on?,Iso-Kukkanen,finnish,"{'answer_start': [-1], 'answer_text': ['']}",\nIso-Kukkanen [1][2] on Päijät-Hämeessä Lahde...,https://fi.wikipedia.org/wiki/Iso-Kukkanen
116014,Kuinka monta jumalaa buddhalaisilla on?,Buddhalaisuus,finnish,"{'answer_start': [-1], 'answer_text': ['']}","Buddhalaisuuden tärkein opetus on, että elämäs...",https://fi.wikipedia.org/wiki/Buddhalaisuus
116031,Miten pitkiä tekstiviestit olivat alunperin ko...,Tekstiviesti,finnish,"{'answer_start': [-1], 'answer_text': ['']}",Tekstiviestejä on käytetty myös vaalivaikuttam...,https://fi.wikipedia.org/wiki/Tekstiviesti
116036,Mitä delirium tremens tarkoittaa?,Delirium tremens,finnish,"{'answer_start': [-1], 'answer_text': ['']}",Alkoholi lamauttaa hermosoluja tehostamalla GA...,https://fi.wikipedia.org/wiki/Delirium%20tremens


In [7]:
def get_answerability(annotations):
    answerable = []
    for annot in annotations:
        if -1 in annot['answer_start']:
            answerable.append(0)
        else:
            answerable.append(1)
    return answerable


train_set['answerable'] = get_answerability(train_set['annotations'].tolist())
validation_set['answerable'] = get_answerability(validation_set['annotations'].tolist())

train_set.columns

Index(['question_text', 'document_title', 'language', 'annotations',
       'document_plaintext', 'document_url', 'answerable'],
      dtype='object')

In [30]:
from transformers import BertTokenizer, BertModel
#%pip install SentencePiece 

model_checkpoint = 'bert-base-multilingual-uncased'

tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
model = BertModel.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
def question_parag_combine(questions, paragraphs):
    """
    This function combines the questions and paragraphs into a single text
    Args:
        questions: list of questions
        paragraphs: list of paragraphs
    Returns:
        list of combined questions and paragraphs
    """
    training_data = []
    for index in range(len(questions)):
        training_data += [questions[index] + "\n" + paragraphs[index]]
        
    return training_data

In [32]:
from datasets import DatasetDict, Dataset

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

def make_df(train_set, validation_set):
  d_q = train_set['question_text'].tolist()
  d_p = train_set['document_plaintext'].tolist()
  training_data = question_parag_combine(d_q,d_p)
  training_labels = train_set['answerable'].tolist()

  d_q = validation_set['question_text'].tolist()
  d_p = validation_set['document_plaintext'].tolist()
  validation_data = question_parag_combine(d_q,d_p)
  validation_labels = validation_set['answerable'].tolist()



  data_set = {}
  sets = [['train',training_data, training_labels], ['val', validation_data, validation_labels]]
  for meta in sets:
      data_set[meta[0]] = {}
      data_set[meta[0]]['text'] = []
      data_set[meta[0]]['labels'] = []
      
      for ind, text in enumerate(meta[1]):
          data_set[meta[0]]['text'].append(text)
          data_set[meta[0]]['labels'].append(meta[2][ind])

          

          
  data_set = DatasetDict({'train':Dataset.from_dict(data_set['train']),
                          'valid':Dataset.from_dict(data_set['val'])\
                        })
  return data_set

data_set = make_df(train_set, validation_set)

data_set

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 116067
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 13325
    })
})

In [33]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    keys = ['attention_mask', 'input_ids']
    concatenated_examples = {k: sum(examples[k], []) for k in keys}
    total_length = len(concatenated_examples[list(keys)[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # this is needed as the used dataset is a subclass of ClassificationDataset, which requires label as a field...
    result["label"] = result["input_ids"].copy()
    result["labels"] = result["input_ids"].copy()
    return result

In [36]:
tokenized_datasets = data_set.map(tokenize_function, batched=True, batch_size=500, num_proc=1, remove_columns=["text"])
tokenized_datasets_lm = tokenized_datasets.map(group_texts, batched=True, batch_size=1000, num_proc=2)
tokenized_datasets_lm = tokenized_datasets_lm.remove_columns(['label'])


  0%|          | 0/233 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets_lm = tokenized_datasets.map(group_texts, batched=True, batch_size=1000, num_proc=2)
tokenized_datasets_lm = tokenized_datasets_lm.remove_columns(['label'])

In [51]:
#model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    #max_steps=300
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [52]:
eng_trainer = Trainer(
    model=eng_model,
    args=training_args,
    train_dataset=lm_eng['train'],
    eval_dataset= lm_eng['valid'],
)

fin_trainer = Trainer(
    model=fin_model,
    args=training_args,
    train_dataset=lm_fin['train'],
    eval_dataset= lm_fin['valid'],
)

jap_trainer = Trainer(
    model=jap_model,
    args=training_args,
    train_dataset=lm_jap['train'],
    eval_dataset= lm_jap['valid'],
)

In [57]:
eng_eval_results = eng_trainer.evaluate()
fin_eval_results = fin_trainer.evaluate()
jap_eval_results = jap_trainer.evaluate()

print(f"Perplexity: {math.exp(eng_eval_results['eval_loss']):.2f}")
print(f"Perplexity: {math.exp(fin_eval_results['eval_loss']):.2f}")
print(f"Perplexity: {math.exp(jap_eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1107
  Batch size = 8


***** Running Evaluation *****
  Num examples = 1614
  Batch size = 8


***** Running Evaluation *****
  Num examples = 1040
  Batch size = 8


Perplexity: 48.74
Perplexity: 91452415747339640111104.00
Perplexity: 41.92


In [58]:
eng_trainer.train()
jap_trainer.train()
fin_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8095
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1012


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [39]:
eng_eval_results = eng_trainer.evaluate()
print(f"Perplexity: {math.exp(eng_eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1107
  Batch size = 8


Perplexity: 37.83


In [36]:
model.save_pretrained("2-epoch_quest_english_finetune")
que

Configuration saved in 2-epoch_quest_english_finetune/config.json
Model weights saved in 2-epoch_quest_english_finetune/pytorch_model.bin


NameError: ignored

In [56]:
# Export contextual representations:

def mean_pooling(model_output, attention_mask):
    # Mean Pooling - Take attention mask into account for correct averaging
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(model_output.size()).float()
    sum_embeddings = torch.sum(model_output * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

encoded = eng_tokenizer(eng_set['train']['text'][100],
                    return_tensors='pt')
encoded = {k:v.to(device) for k, v in encoded.items()}

model_output = eng_model(**encoded, output_hidden_states=True, return_dict=True)

print(model_output.keys())
print(len(model_output['hidden_states'])) # contextual representations of separate words from each of the 6 layes
print(model_output['hidden_states'][-1].shape) # last layer with contextual representations (batch_size x num words x representation dim)

# Aggregate all the representations into one
mean_model_output = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])

mean_model_output.shape

odict_keys(['logits', 'past_key_values', 'hidden_states'])
7
torch.Size([1, 65, 768])


torch.Size([1, 768])

In [19]:
index = 0
input_string = d_q[index]
answer_string = d_p[index]
input_ids = tokenizer.encode(input_string, return_tensors='pt').cuda()

print(f'Input: {input_string}')
print(f'Answer: {answer_string}')
print('------------------------')
greedy_output = model.generate(input_ids, max_length=50, do_sample=True, top_k=0, top_p=0.2)
print("Tok-p Nucleus Sampling:")
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True, do_sample=True))
print('------------------------')

greedy_output = model.generate(input_ids, max_length=50, do_sample=True, top_k=0, top_p=0.92)
print("Tok-p Nucleus Sampling:")
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True, do_sample=True))
print('------------------------')

greedy_output = model.generate(input_ids, max_length=50, do_sample=True, top_k=0, top_p=0.99)
print("Tok-p Nucleus Sampling:")
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True, do_sample=True))
print('------------------------')


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: What is a way to increase your wound healing speed?
Answer: Wound care encourages and speeds wound healing via cleaning and protection from reinjury or infection. Depending on each patient's needs, it can range from the simplest first aid to entire nursing specialties such as wound, ostomy, and continence nursing and burn center care.
------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tok-p Nucleus Sampling:
What is a way to increase your wound healing speed?
In the case of the C. difficile, the C. difficile is a soft tissue that is covered by a layer of soft tissue. The soft tissue is covered by a
------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tok-p Nucleus Sampling:
What is a way to increase your wound healing speed?
When did the NFL lockout end?
By early May, more than 1,800 NFL teams signed offers to enter the NFL, including the University of Southern California, the New York Giants,
------------------------
Tok-p Nucleus Sampling:
What is a way to increase your wound healing speed?
Immunals are a special type of granular compartments with layers that extend from the outer punctus to the internal organs. The interlocked compartments consist of magnetic ducts connected continuously
------------------------


OLD STUFF

In [ ]:
data_set['train']['text'][100]


In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from datasets import load_metric
metric = load_metric('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    #learning_rate=2e-5,
    #weight_decay=0.01,
    num_train_epochs=2,
    do_train=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(200)),
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
)